In [2]:
import json
import pickle
import collections
import random

In [13]:
gen_sample_size = 30000
stnd_score = 80 # vae:55 , seq:80
json_filename = "seq2seq_aug_korquad_train.json"
pkl_filename = "seq2seq_aug_korquad_train_f1_scores.pkl"
final_aug_file_name = 'aug_korquad_train_sample_less100.json'

# json_filename = "aug_train_wQA.json"
# pkl_filename = "aug_train_wQA_f1_scores.pkl"
# final_aug_file_name = 'aug_train_wQA_sample.json'

# json_filename = "KorQuAD_v1.0_train_wQA_f1_BEST_x3.json"
# pkl_filename = "f1_scores_vae_f1_best_x3.pkl"
# final_aug_file_name = 'KorQuAD_v1.0_train_wQA_f1_BEST_x3_sample_less100.json'

In [14]:
with open(pkl_filename, 'rb') as file:
    data = pickle.load(file)

sorted_data = sorted(data.items(), key=lambda kv: kv[1])
sorted_data = collections.OrderedDict(sorted_data)

In [15]:
######################### cut score 100 seperately #########################
original_qas_list = dict()
gen_qas_list = dict()
score100_qas_list = dict()
underscore_qas_list = []

for (key, value) in sorted_data.items():
    if(not str(key).startswith('9')):
        original_qas_list[key] = value
    else:        
        if(value == 100):
            score100_qas_list[key] = value
        elif(value >= stnd_score):
            gen_qas_list[key] = value        
        else:
            underscore_qas_list.append(key)
        
gen_qas_size = len(gen_qas_list)
score100_qas_size = len(score100_qas_list)
original_qas_size = len(original_qas_list)
underscore_qas_size = len(underscore_qas_list)
print('score 합격 qas: %d' % gen_qas_size)
print('score 100 qas: %d' % score100_qas_size)
print('score 불합격 qas: %d' % underscore_qas_size)
print('original qas size: %d' % original_qas_size)

# random sample from 100 score list

sample_score100_size = gen_sample_size - gen_qas_size
minus_score100_size = score100_qas_size - sample_score100_size
print('삭제할 score 100 sample size : %d ' % minus_score100_size)
sample_minus_score100_qas_list = random.sample(list(score100_qas_list), minus_score100_size)
sample_minus_qas_list = sample_minus_score100_qas_list + underscore_qas_list

print('최종 qas size(over 80~99 + 100 + original):  %d + %d + %d = %d' % (gen_qas_size, sample_score100_size, original_qas_size, (gen_qas_size+sample_score100_size+original_qas_size)))

score 합격 qas: 2486
score 100 qas: 51968
score 불합격 qas: 12758
original qas size: 60407
삭제할 score 100 sample size : 24454 
최종 qas size(over 80~99 + 100 + original):  2486 + 27514 + 60407 = 90407


In [16]:
with open(json_filename) as json_file:
    data_json = json.load(json_file)
    
datai = data_json['data']

In [17]:
data_list_size = len(datai)
minus_cnt = 0
question_size = 0

data_index = 0
while(True):
    if(data_index == data_list_size):
        break

    paragraph_list = datai[data_index]['paragraphs']    
    paragraph_list_size = len(paragraph_list)
    paragraph_index = 0
    while(True):
        if(paragraph_index == paragraph_list_size):
            break
        
        paragA = paragraph_list[paragraph_index]        
        qas_list = paragA['qas']
        qlen = len(qas_list)
        index = 0
        while(True):            
            if(qlen == index):
                break
                        
            if(qas_list[index]['id'] in sample_minus_qas_list):
                qas_list.pop(index)
                minus_cnt += 1                
                qlen -= 1
            else:
                question_size += 1
                if(question_size % 10000 == 0):
                    print("-> parsed : " + str(question_size))
                index +=1
        
        if(qlen == 0):
#             print('is empty paragraph')
            paragraph_list.pop(paragraph_index)
            paragraph_list_size -= 1
        else:
            paragraph_index+=1
    
    if(paragraph_list_size==0):
#         print('is empty data')
        datai.pop(data_index)        
        data_list_size -= 1
    else:
        data_index+=1

print('minus count: %d' % minus_cnt)
print('Generate QAS size : %d (original:%d / generation:%d)' % (question_size, original_qas_size, gen_sample_size))

new_sample_data = dict()
new_sample_data["version"] = "KorQuAD_v1.0_sample_train"
new_sample_data["data"] = datai

with open(final_aug_file_name, "w") as writer:
    writer.write(json.dumps(new_sample_data, indent=4) + "\n")

-> parsed : 10000
-> parsed : 20000
-> parsed : 30000
-> parsed : 40000
-> parsed : 50000
-> parsed : 60000
-> parsed : 70000
-> parsed : 80000
-> parsed : 90000
minus count: 37212
Generate QAS size : 90407 (original:60407 / generation:30000)
